In [ ]:
# ============================================================
# Global Summary Table — Country-level Aggregation
# RF city points → country summary (area + population)
#
# Inputs (relative to repo root):
#   - 4_Figures_Tables/AllCities_Points_new.gpkg
#   - 4_Figures_Tables/country_ISOcodes_new.csv
#
# Output (written once, then manually cleaned):
#   - 4_Figures_Tables/Tables/CountrySummary_RF_raw.csv
#
# NOTE:
#   The first run may contain rows with missing ISO3 codes.
#   These ISO codes are checked and filled manually.
#   The cleaned, final table is then saved/uploaded as:
#   - 4_Figures_Tables/Tables/CountrySummary_RF.csv
# ============================================================

In [ ]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
# -------- CONFIG (repo-relative paths) --------
REPO_ROOT   = Path.cwd()

POINTS_GPKG = REPO_ROOT / "4_Figures_Tables" / "AllCities_Points_new.gpkg"
ISO_CSV     = REPO_ROOT / "4_Figures_Tables" / "country_ISOcodes_new.csv"
TABLE_DIR   = REPO_ROOT / "4_Figures_Tables" / "Tables"
TABLE_DIR.mkdir(parents=True, exist_ok=True)

# Raw output from this script (before manual ISO fixes)
OUT_CSV_RAW = TABLE_DIR / "CountrySummary_RF_raw.csv"

print("📂 Points GPKG :", POINTS_GPKG)
print("📂 ISO table   :", ISO_CSV)
print("📂 Out (raw)   :", OUT_CSV_RAW)

# -------- LOAD DATA --------
print("\n📥 Loading files...")
gdf = gpd.read_file(POINTS_GPKG)
iso_df = pd.read_csv(ISO_CSV)

# Normalize country names for matching
gdf["CTR_MN_NM_norm"] = gdf["CTR_MN_NM"].str.lower().str.strip()
iso_df["Country_norm"] = iso_df["Country"].str.lower().str.strip()

In [ ]:
# -------- AGGREGATE BY COUNTRY --------
agg = (
    gdf.groupby("CTR_MN_NM_norm")
      .agg(
          Country=("CTR_MN_NM", "first"),
          Region=("REG1_GHSL", "first"),
          Total_Area_km2=("AREA_TOTAL", lambda x: np.sum(x) / 1e6),
          Deprived_Area_km2=("AREA_DEPRIVED", lambda x: np.sum(x) / 1e6),
          Total_Pop_M=("POP_TOTAL", lambda x: np.sum(x) / 1e6),
          Deprived_Pop_M=("DEPRIVED_POP", lambda x: np.sum(x) / 1e6),
      )
      .reset_index(drop=True)
)

# -------- MERGE ISO CODES --------
agg["Country_norm"] = agg["Country"].str.lower().str.strip()

agg = (
    agg.merge(
        iso_df[["Country_norm", "ISO"]],
        how="left",
        on="Country_norm"
    )
    .drop(columns="Country_norm")
)

# Reorder columns
agg = agg[[
    "Region", "ISO", "Country",
    "Total_Area_km2", "Deprived_Area_km2",
    "Total_Pop_M", "Deprived_Pop_M"
]]

# -------- SAVE RAW OUTPUT --------
agg.to_csv(OUT_CSV_RAW, index=False, float_format="%.3f")

print(f"\n✅ Saved RAW country-level summary to:\n{OUT_CSV_RAW}")
print(f"🌍 Total countries summarized: {len(agg)}")

# List countries missing ISO (to be fixed manually)
missing_iso = agg[agg["ISO"].isna()]
if not missing_iso.empty:
    print("\n⚠️ Countries missing ISO code (to fix manually):")
    print(missing_iso[["Country", "Region"]].to_string(index=False))
else:
    print("\n✅ All countries have ISO codes.")

# Quick preview
print("\nSample rows:")
print(agg.head(10).to_string(index=False))

In [ ]:
# ----------------------------------------------------------------
# After running this script once:
#   1. Open CountrySummary_RF_raw.csv
#   2. Manually fill / correct any missing ISO codes.
#   3. The cleaned version is saved as:
#        4_Figures_Tables/Tables/CountrySummary_RF.csv
# ----------------------------------------------------------------